<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Libraries" data-toc-modified-id="Libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Libraries</a></span></li><li><span><a href="#Office-Example" data-toc-modified-id="Office-Example-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Office Example</a></span><ul class="toc-item"><li><span><a href="#Artificial-Example" data-toc-modified-id="Artificial-Example-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Artificial Example</a></span></li><li><span><a href="#Problema-de-Optimización" data-toc-modified-id="Problema-de-Optimización-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Problema de Optimización</a></span></li></ul></li></ul></div>

## Libraries

In [1]:
# Optimization tools
from __future__ import print_function
from ortools.linear_solver import pywraplp

# Data Science
import pandas as pd
import numpy as np

## Office Example

### Artificial Example

In [2]:
# Personas
people = pd.DataFrame()
num_people = 1000

people_names = []
for i in range(num_people):
    people_names.append("Person " + str(i+1))
    
people["Name"] = people_names
people["Flexibility"] = np.random.randint(1, 6, num_people)


# Offices
offices = pd.DataFrame()
num_offices = 6

nombres_oficinas = []
for i in range(num_offices):
    nombres_oficinas.append("Office " + str(i+1))

offices["Name"] = nombres_oficinas
offices["Capacity"] = np.random.randint(100,300,num_offices)
offices["Fixed Cost"] = np.random.randint(2000,10000,num_offices)
offices["Variable Cost"] = np.random.randint(20,80,num_offices)

In [3]:
people

,Name,Flexibility
0,Person 1,4
1,Person 2,5
2,Person 3,5
3,Person 4,3
4,Person 5,5
...,...,...
995,Person 996,1
996,Person 997,2
997,Person 998,2
998,Person 999,3


In [4]:
offices

,Name,Capacity,Fixed Cost,Variable Cost
0,Office 1,230,9899,70
1,Office 2,284,3809,46
2,Office 3,218,7045,38
3,Office 4,153,8528,56
4,Office 5,206,3356,53
5,Office 6,178,5966,62


### Problema de Optimización

In [5]:
# Integer programming solver
solver = pywraplp.Solver.CreateSolver('CBC')

# Creamos un arreglo 3-D
dim_1 = people.shape[0] # Number of people
dim_2 = 5 # Number of days
dim_3 = offices.shape[0] # Number of offices

# Cont
contador = 1

# People variables
variables_personas = [[[0 for k in range(dim_3)] for j in range(dim_2)] for i in range(dim_1)]

for i in range(dim_1):
    for j in range(dim_2):
        for k in range(dim_3):
            variables_personas[i][j][k] = solver.IntVar(0.0, 1.0, 'x[%i]' % contador)
            contador += 1
            
# Variables de las oficinas
variables_oficina = [0 for i in range(dim_3)]

for i in range(dim_3):
    variables_oficina[i] = solver.IntVar(0.0, 1.0, 'x[%i]' % contador)
    contador += 1
            
            
# Restriccion 1 (Las personas deben ir al menos una cantidad de días) 
constraint_1 = [_ for i in range(dim_1)]
for persona in range(dim_1): # Para toda persona
    dias_trabajados = int(people["Flexibility"][persona])
    constraint_1[persona] = solver.Constraint(dias_trabajados, dias_trabajados)
    for dia in range(dim_2):
        for off in range(dim_3):
            constraint_1[persona].SetCoefficient(variables_personas[persona][dia][off], 1)
            
# Restriccion 2 (No exceder la capacidad) 
constraint_2 = [[_ for k in range(dim_3)] for j in range(dim_2)] 
for dia in range(dim_2):
    for off in range(dim_3):
        capacidad_oficina = int(offices["Capacity"][off])
        constraint_2[dia][off] = solver.Constraint(-solver.infinity(), 0)
        constraint_2[dia][off].SetCoefficient(variables_oficina[off], -capacidad_oficina)
        for persona in range(dim_1):
            constraint_2[dia][off].SetCoefficient(variables_personas[persona][dia][off], 1)
        
# Restriccion 3 (Una persona no puede estar en dos lugares a la vez) 
constraint_3 = [[_ for k in range(dim_2)] for j in range(dim_1)] 
for persona in range(dim_1): # Para toda persona
    for dia in range(dim_2): # Para todo día
        constraint_3[persona][dia] = solver.Constraint(-solver.infinity(), 1)
        for off in range(dim_3):
            constraint_3[persona][dia].SetCoefficient(variables_personas[persona][dia][off], 1)
              
# Función Objetivo
objective = solver.Objective()
for off in range(dim_3):
    objective.SetCoefficient(variables_oficina[off], int(offices["Fixed Cost"][off]))
    
    
for persona in range(dim_1):
    for dia in range(dim_2):
        for off in range(dim_3):
            objective.SetCoefficient(variables_personas[persona][dia][off], int(offices["Fixed Cost"][off]))
            
objective.SetMinimization()

In [6]:
# Solver del problema
solver.Solve()

# Valor de la función objetivo
print('Maximum objective function value = %d' % solver.Objective().Value())

# Que oficinas se necesitan y cuales no
for var in variables_oficina:
    print('%s = %d' % (var.name(), var.solution_value()))

Maximum objective function value = 12482055
x[30001] = 0
x[30002] = 1
x[30003] = 0
x[30004] = 0
x[30005] = 1
x[30006] = 1


In [7]:
# Output

lista_personas = []
lista_dias = []
lista_oficina = [] 

for i in range(dim_1):
    for j in range(dim_2):
        for k in range(dim_3):
            if variables_personas[i][j][k].solution_value() != 0:
                lista_personas.append(people["Name"][i])
                lista_dias.append(j)
                lista_oficina.append(offices["Name"][k])
                

output_optimizacion = pd.DataFrame()

output_optimizacion["Name"] = lista_personas
output_optimizacion["Day"] = lista_dias
output_optimizacion["Place"] = lista_oficina

output_optimizacion= output_optimizacion.sort_values(by=["Day","Name"])

output_optimizacion.head()

,Name,Day,Place
0,Person 1,0,Office 5
35,Person 10,0,Office 6
3052,Person 1000,0,Office 2
300,Person 101,0,Office 2
305,Person 102,0,Office 2


In [8]:
people[people["Name"]=="Person 31"]

,Name,Flexibility
30,Person 31,1


In [9]:
output_optimizacion[output_optimizacion["Name"]=="Person 31"]

,Name,Day,Place
91,Person 31,1,Office 6


In [10]:
office_ocupancy = []
for off in output_optimizacion["Place"].unique():
    day_occupancy = []
    for day in output_optimizacion["Day"].unique():
        data_temp = output_optimizacion[(output_optimizacion["Day"]==day) & (output_optimizacion["Place"]==off)]
        day_occupancy.append(data_temp.shape[0])
    office_ocupancy.append(day_occupancy)

day_names = {0:"Monday",1:"Tuesday",2:"Wednesday",3:"Thursday",4:"Friday"}
occupancy = pd.DataFrame(data=office_ocupancy,columns=[day_names[a] for a in output_optimizacion["Day"].unique()],index=output_optimizacion["Place"].unique())
occupancy = occupancy.reset_index().rename(columns={"index":"Name"}).sort_values(by="Name").reset_index(drop=True)
resumen_oficinas = occupancy.merge(offices)
resumen_oficinas

,Name,Monday,Tuesday,Wednesday,Thursday,Friday,Capacity,Fixed Cost,Variable Cost
0,Office 2,284,284,284,284,284,284,3809,46
1,Office 5,206,206,206,206,206,206,3356,53
2,Office 6,121,121,120,121,121,178,5966,62
